In [24]:
# imports
# Import necessary libraries
import requests  # For API requests
import pandas as pd  # For data handling
from dotenv  import load_dotenv # For loading .env variables
import os # For accessing environment variables

# Load environment variables from .env file
load_dotenv()

# Load API keys
FOURSQUARE_API_KEY = os.getenv("FOURSQUARE_API_KEY")

# Verify API keys
if FOURSQUARE_API_KEY:
    print("API keys loaded successfully!")
else:
    print("Error: API keys not loaded. Check your .env file.")

API keys loaded successfully!


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [35]:

# Foursquare API setup
foursquare_url = "https://api.foursquare.com/v3/places/search"
headers = {"Authorization": FOURSQUARE_API_KEY,
           "accept":"application/json"}

def query_foursquare(latitude, longitude, radius=1000, category="restaurants,bars"):
    """
    Query Foursquare API for POIs near a given latitude and longitude.
    """
    params = {
        "ll": f"{latitude},{longitude}",  # Latitude and longitude
        "radius": radius,  # Search radius in meters
        "categories": category,  # Restaurants/Bars
        "limit": 50  # Number of results
    }
    try:
        response = requests.get(foursquare_url, headers=headers, params=params, timeout = 300)
        response.raise_for_status()
        return response.json().get("results", [])
    except requests.exceptions.RequestException as e:
        print(f"Foursquare API Error: {e}")
    return []




Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [36]:
# Load bike station data
station_df = pd.read_csv("luxembourg_bike_stations.csv")

# Store Foursquare data
foursquare_data = []

# Loop through bike stations and query Foursquare
for _, row in station_df.iterrows():
    station_name = row["station_name"]
    latitude = row["latitude"]
    longitude = row["longitude"]
    
    fsq_results = query_foursquare(latitude, longitude)
    for result in fsq_results:
        if "name" in result and "geocodes" in result and "main" in result["geocodes"]:
            foursquare_data.append({
                "station_name": station_name,
                "poi_name": result.get("name"),
                "category": result["categories"][0]["name"] if result.get("categories") else None,
                "latitude": result["geocodes"]["main"]["latitude"],
                "longitude": result["geocodes"]["main"]["longitude"]
            })


Foursquare API Error: 400 Client Error: Bad Request for url: https://api.foursquare.com/v3/places/search?ll=49.615901%2C5.993194&radius=1000&categories=restaurants%2Cbars&limit=50
Foursquare API Error: 400 Client Error: Bad Request for url: https://api.foursquare.com/v3/places/search?ll=49.569178%2C6.08175&radius=1000&categories=restaurants%2Cbars&limit=50
Foursquare API Error: 400 Client Error: Bad Request for url: https://api.foursquare.com/v3/places/search?ll=49.59%2C6.1172&radius=1000&categories=restaurants%2Cbars&limit=50
Foursquare API Error: 400 Client Error: Bad Request for url: https://api.foursquare.com/v3/places/search?ll=49.60951%2C6.128503&radius=1000&categories=restaurants%2Cbars&limit=50
Foursquare API Error: 400 Client Error: Bad Request for url: https://api.foursquare.com/v3/places/search?ll=49.649823%2C6.225796&radius=1000&categories=restaurants%2Cbars&limit=50
Foursquare API Error: 400 Client Error: Bad Request for url: https://api.foursquare.com/v3/places/search?ll=

Put your parsed results into a DataFrame

In [31]:
 ##Create a DataFrame for Foursquare data
foursquare_df = pd.DataFrame(foursquare_data)
print("Foursquare DataFrame:")
print(foursquare_df.head())

Foursquare DataFrame:
                     station_name                poi_name  \
0  #00126-SALLES DES FÊTES HOLZEM       Restaurant Papaya   
1                    #00109-FOYER  Burger King Leudelange   
2                    #00109-FOYER              Spackelter   
3                    #00109-FOYER    Les Espaces Réunions   
4                    #00109-FOYER      Il Punto Delizioso   

               category   latitude  longitude  
0      Sushi Restaurant  49.616679   5.993284  
1  Fast Food Restaurant  49.567216   6.079689  
2                 Diner  49.562383   6.067011  
3     French Restaurant  49.563044   6.072636  
4              Pizzeria  49.565284   6.080453  


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
# Yelp API setup
# Load environment variables from .env file
import requests
from dotenv import load_dotenv
import os

load_dotenv()

# Load API keys
YELP_API_KEY = os.getenv("YELP_API_KEY")

# Verify API keys
if YELP_API_KEY:
    print("API keys loaded successfully!")
else:
    print("Error: API keys not loaded. Check your .env file.")
yelp_url = "https://api.yelp.com/v3/businesses/search"
yelp_headers = {"Authorization": f"Bearer {YELP_API_KEY}"}

def query_yelp(latitude, longitude, radius=1000, categories="restaurants,bars"):
    """
    Query Yelp API for POIs near a given latitude and longitude.
    """
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "categories": categories,
        "limit": 50  # Number of results
    }
    try:
        response = requests.get(yelp_url, headers=yelp_headers, params=params)
        response.raise_for_status()
        return response.json().get("businesses", [])
    except requests.exceptions.RequestException as e:
        print(f"Yelp API Error: {e}")
        return []




API keys loaded successfully!


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [32]:
 ##Load bike station data
import pandas as pd
station_df = pd.read_csv("luxembourg_bike_stations.csv")

## Store Yelp data
yelp_data = []

# Loop through bike stations and query Yelp
for _, row in station_df.iterrows():
    station_name = row["station_name"]
    latitude = row["latitude"]
    longitude = row["longitude"]
    
    yelp_results = query_yelp(latitude, longitude)
    for business in yelp_results:
        if "name" in business and "coordinates" in business:
            yelp_data.append({
                "station_name": station_name,
                "poi_name": business.get("name"),
                "category": business["categories"][0]["title"] if business.get("categories") else None,
                "rating": business.get("rating"),
                "review_count": business.get("review_count"),
                "latitude": business["coordinates"]["latitude"],
                "longitude": business["coordinates"]["longitude"]
            })

Put your parsed results into a DataFrame

In [33]:
## Create a DataFrame for Yelp data
yelp_df = pd.DataFrame(yelp_data)
print("Yelp DataFrame:")
print(yelp_df.head())


Yelp DataFrame:
            station_name                     poi_name    category  rating  \
0       #00013-GELLE FRA  Brauerei - Big Beer Company  Brasseries     3.4   
1     #00077-FUNICULAIRE  Brauerei - Big Beer Company  Brasseries     3.4   
2  #00038-HELLEG GEESCHT  Brauerei - Big Beer Company  Brasseries     3.4   
3          #00004-ROCADE  Brauerei - Big Beer Company  Brasseries     3.4   
4          #00016-AMELIE  Brauerei - Big Beer Company  Brasseries     3.4   

   review_count   latitude  longitude  
0             5  49.612585   6.142349  
1             5  49.612585   6.142349  
2             5  49.612585   6.142349  
3             5  49.612585   6.142349  
4             5  49.612585   6.142349  


# Comparing Results

In [34]:
 ## load dataframe yelp and foursquare
yelp_df = pd.DataFrame(yelp_data)
## load foursquare dataframe
foursquare_df = pd.DataFrame(foursquare_data)



# Compare the number of POIs retrieved
print(f"Number of POIs from Foursquare: {len(foursquare_df)}")
print(f"Number of POIs from Yelp: {len(yelp_df)}")

# Compare data richness (Yelp: Ratings and Reviews)
if not yelp_df.empty:
    avg_rating = yelp_df["rating"].mean()
    avg_reviews = yelp_df["review_count"].mean()
    print(f"\nYelp - Average Rating: {avg_rating:.2f}")
    print(f"Yelp - Average Reviews: {avg_reviews:.2f}")

# Observations
if len(foursquare_df) > len(yelp_df):
    print("\nFoursquare provided more POIs.")
else:
    print("\nYelp provided more POIs.")

# Save both DataFrames for further use
foursquare_df.to_csv("foursquare_pois.csv", index=False)
yelp_df.to_csv("yelp_pois.csv", index=False)
print("\nData saved to CSV files: foursquare_pois.csv and yelp_pois.csv")


Number of POIs from Foursquare: 6698
Number of POIs from Yelp: 37

Yelp - Average Rating: 3.40
Yelp - Average Reviews: 5.00

Foursquare provided more POIs.

Data saved to CSV files: foursquare_pois.csv and yelp_pois.csv


Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [22]:
# Filter Yelp DataFrame for restaurants only
restaurants_yelp = yelp_df[yelp_df["category"].str.contains("restaurant", case=False, na=False)]

# Sort by rating and get the top 10 restaurants
top_10_restaurants = restaurants_yelp.sort_values(by="rating", ascending=False).head(10)

print("Top 10 Restaurants by Rating:")
print(top_10_restaurants[["poi_name", "rating", "review_count", "station_name"]])


Top 10 Restaurants by Rating:
Empty DataFrame
Columns: [poi_name, rating, review_count, station_name]
Index: []
